In [ ]:
import tensorflow as tf
global_step = tf.Variable(0)
# 通过eponentiial_decay函数生成学习率
learning_rate = tf.train.exponential_decay(0.1, global_step,100,0.96,staircase = True)
#使用指数衰减的学习率，在minimize函数中传入global_step 将自动更新
learning_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

In [ ]:
#学习率的设置
w = tf.Variable(tf.random_normal([2,1],stddev=1,seed =1))
y= tf.matmul(x,w)

loss =tf.reduce_mean(tf.square(y_ -y)) + tf.contrib.layers.l2_regularizer(lambda)(w)

In [8]:
# 正则化防止过拟合
weights = tf.constant([[1.,-2.],[-3.,4.]])
with tf.Session() as sess:
    print(sess.run(tf.contrib.layers.l1_regularizer(0.5)(weights)))

with tf.Session() as sess:
    print(sess.run(tf.contrib.layers.l2_regularizer(0.5)(weights)))


5.0
7.5


In [30]:
# 5层的L2正则化的损失函数
import tensorflow as tf
from numpy.random import RandomState
import numpy as np
#获取一层神经网络边上的权重，并将这个权重的L2正则化损失加入名称为'losses'的集合中
def get_weight(shape,l2_weight):
#生成一个变量
    var = tf.Variable(tf.random_normal(shape),dtype=tf.float32)
    # add_to_collection 函数将这个新生成变量的L2正则化损失项加入集合
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(l2_weight)(var))
    return var


x = tf.placeholder(tf.float32,shape=(None,2))
y_ = tf.placeholder(tf.float32,shape=(None,1))

#生成模拟数据
rdm = RandomState(1)
dataset_size =128
X = rdm.rand(dataset_size,2)
# Y = [[int(x1+x2<1)] for (x1,x2) in X]
Y = np.random.randint(0,2,128)
Y =[[x] for x in Y]
batch_size = 8

# 定义了每一层网络节点的个数
layer_dimension = [2,10,10,10,1]

#神经网络的层数
n_layers = len(layer_dimension)

# 这个变量维护向前传播时最深层的节点，开始的时候就是输入层
cur_layer = x
# 当前层的节点个数
in_dimension =layer_dimension[0]

#通过一个循环来生成5层的全连接的神经网络结构
for i in range(1,n_layers):
    #lay_dimension[i]为下一层的节点数
    out_dimension = layer_dimension[i]
    #生成当前层中权重的变量，并将这个变量的L2正则化加入计算图上的集合
    weight = get_weight([in_dimension,out_dimension],0.001)
    bias = tf.Variable(tf.constant(0.1,shape=[out_dimension]))
    cur_layer = tf.nn.relu(tf.matmul(cur_layer,weight)+bias)
    in_dimension = out_dimension
    
# 在定义神经网络前向传播的同时已经将所有的L2正则化损失加入到图上的集合
#这里只需要计算刻画模型在训练数据上表现的损失函数
mse_loss =tf.reduce_mean(tf.square(y_ -cur_layer))
# 将均方误差函数加入到损失函数
tf.add_to_collection('losses',mse_loss)
#get_collection返回一个列表，这个列表是所有这个集合中的元素，这些元素是损失函数的不同部分，将它们加起来可以得到最终的损失函数
loss = tf.add_n(tf.get_collection('losses'))
train_step = tf.train.AdamOptimizer(0.01).minimize(loss)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    STEPS = 100
    for i in range(STEPS):
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        print(loss)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_12' with dtype float and shape [?,2]
	 [[Node: Placeholder_12 = Placeholder[dtype=DT_FLOAT, shape=[?,2], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_12', defined at:
  File "C:\Program Files (x86)\software\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Program Files (x86)\software\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files (x86)\software\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Program Files (x86)\software\Anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Program Files (x86)\software\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Program Files (x86)\software\Anaconda\l

In [36]:
#滑动平均模型
import tensorflow as tf
# 定义一个变量用于计算滑动平均
v1 = tf.Variable(0,dtype = tf.float32)
# 模拟神经网络中迭代的次数，可以用于动态控制衰减率
step = tf.Variable(0,trainable=False)
# 定义一个滑动平均类，初始化时给了衰减率（0.99）和控制衰减率的变量step
ema = tf.train.ExponentialMovingAverage(0.99,step)
# 定义一个更新变量滑动平均的操作，这里是一个列表，每执行一次操作，这个列表中的变量都会更新
maintain_average_op = ema.apply([v1])

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run([v1,ema.average(v1)]))
#     更新变量v1的值到5
    sess.run(tf.assign(v1,5))
    sess.run(maintain_average_op)
    
    print(sess.run([v1,ema.average(v1)]))
    
    sess.run(tf.assign(step,10000))
    sess.run(tf.assign(v1,10))
    sess.run(maintain_average_op)
    print(sess.run([v1,ema.average(v1)]))
    
    sess.run(maintain_average_op)
    print(sess.run([v1,ema.average(v1)]))


[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
